In [1]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession


class PSTool:
    def pyspark_session(self, host_location):
        """
        Creates and returns spark session object
        """
        sc = SparkContext(host_location)  # Create spark context
        spark = SparkSession(sc)  # Create session
        return spark

    def file_loader(self, path, delim, spark_obj):
        data = spark_obj.read.options(delimiter=delim).csv(path)  # load file
        return data


if __name__ == "__main__":
    # Instanciate object
    pstool = PSTool()
    # start session
    spk = pstool.pyspark_session('local')
    # load data
    path = 'all_bacilli_subset.tsv'
    df=pstool.file_loader(path, '\t', spk)

22/06/17 14:56:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
df

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string]

In [8]:
# 'Protein _accession', 'Sequence_MD5_digest', 'Sequence_length', 'Analysis',
# 'Signature_accession', 'Signature_description', 'Start location', 'Stop location',
# 'Score', 'Status', 'Date', 'InterPro annotations', 'InterPro annotations' - description (e.g. BRCA2 repeat)
#     (GO annotations (e.g. GO:0005515) - optional column; only displayed if –goterms option is switched on)
#     (Pathways annotations (e.g. REACT_71) - optional column; only displayed if –pathways option is switched on)

The TSV format presents the match data in columns as follows:

    1. Protein accession (e.g. P51587)
    2. Sequence MD5 digest (e.g. 14086411a2cdf1c4cba63020e1622579)
    3. Sequence length (e.g. 3418)
    4. Analysis (e.g. Pfam / PRINTS / Gene3D)
    5. Signature accession (e.g. PF09103 / G3DSA:2.40.50.140)
    6. Signature description (e.g. BRCA2 repeat profile)
    7. Start location
    8. Stop location
    9. Score - is the e-value (or score) of the match reported by member database method (e.g. 3.1E-52)
    10. Status - is the status of the match (T: true)
    11. Date - is the date of the run
    12. InterPro annotations - accession (e.g. IPR002093)
    13. InterPro annotations - description (e.g. BRCA2 repeat)
    14. (GO annotations (e.g. GO:0005515) - optional column; only displayed if –goterms option is switched on)
    15. (Pathways annotations (e.g. REACT_71)-optional column; only displayed if –pathways option is switched on)


# Question 1
How many distinct protein annotations are found in the dataset? I.e. how many distinc InterPRO numbers are there?

In [15]:
InterPRO_distinc = df.select("_c11").distinct().count()
print(InterPRO_distinc)

61


# Question 2
How many annotations does a protein have on average?

In [20]:
total_annot = df.select("_c11").count()
total_annot

200

In [21]:
avg_annot = total_annot/InterPRO_distinc
print(avg_annot)

3.278688524590164


# Question 3
What is the most common GO Term found?

In [ ]:
df.groupBy('state').count().show()

In [30]:
# df.groupBy("_c13").count().show()

In [33]:
from pyspark.sql.functions import split

In [37]:
df1 = df.withColumn('_c13_GO1', split(df['_c13'], '-').getItem(0)) \
       .withColumn('_c13_GO2', split(df['_c13'], '-').getItem(1))

In [42]:
GO1 = df1.select('_c13_GO1')
GO2 = df1.select('_c13_GO2')

In [43]:
type(GO1)

pyspark.sql.dataframe.DataFrame